In [14]:
import numpy as np
import pandas as pd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [4]:
bike_trips = pd.read_csv('D:\\Capstone Data\\2014citibike\\2014-01 - Citi Bike trip data.csv')

In [15]:
regions = pd.read_csv('D:\\Capstone Data\\taxizones.csv')

In [34]:

def extract_bike_stations(bike_trips, region_polygons, distinct_stations = {'station id':[], 'station name':[], 'station latitude':[], 'station longitude':[], 'station zone':[]}):
    '''
    Function to extract the distinct bike locations from citi bike trips data, and attribute each station a zone from the taxi zones

    Args:
        bike_trips (pd.DataFrame): DataFrame with bike trip data
        region_polygons (dict): dictionary with taxi zone id's as the keys and polygon objects as the values
        distinct_stations (dict): dictionary containing stations and their information, defaults to blank dictionary structure

    Returns:
        dict: A processed version of distinct_stations, with the addition of any newly encountered station id's
    '''

    for _, row in bike_trips.iterrows():
        if row.loc['start station id'] not in distinct_stations['station id']:
            distinct_stations['station id'].append(row.loc['start station id'])
            distinct_stations['station name'].append(row.loc['start station name'])
            distinct_stations['station latitude'].append(row.loc['start station latitude'])
            distinct_stations['station longitude'].append(row.loc['start station longitude'])

            s_point = Point(row.loc['start station longitude'], row.loc['start station latitude'])
            zone_set = False
            for zone, poly in region_polygons.items():
                if poly.contains(s_point):
                    distinct_stations['station zone'].append(zone)
                    zone_set = True
                    break
            if not zone_set:
                distinct_stations['station zone'].append(np.nan)
        if row.loc['end station id'] not in distinct_stations['station id']:
            distinct_stations['station id'].append(row.loc['end station id'])
            distinct_stations['station name'].append(row.loc['end station name'])
            distinct_stations['station latitude'].append(row.loc['end station latitude'])
            distinct_stations['station longitude'].append(row.loc['end station longitude'])

            e_point = Point(row.loc['end station longitude'], row.loc['end station latitude'])
            zone_set = False
            for zone, poly in region_polygons.items():
                if poly.contains(e_point):
                    distinct_stations['station zone'].append(zone)
                    zone_set = True
                    break
            if not zone_set:
                distinct_stations['station zone'].append(np.nan)
    
    return distinct_stations

In [35]:
region_polygons = {}

for _, row in regions.iterrows():
    poly = eval(row.loc['the_geom'])['coordinates'][0][0]
    region_polygons[row.loc['location_id']] = Polygon(poly)


distinct_stations = extract_bike_stations(bike_trips, region_polygons)

In [36]:
for (key, items) in distinct_stations.items():
    print(key, len(items))

station id 329
station name 329
station latitude 329
station longitude 329
station zone 329


In [37]:
bike_stations = pd.DataFrame(distinct_stations)
bike_stations.set_index('station id', inplace=True)

In [39]:
bike_stations[bike_stations['station zone']==np.nan]

,station name,station latitude,station longitude,station zone
station id,,,,


In [40]:
bike_stations.head()

,station name,station latitude,station longitude,station zone
station id,,,,
2009,Catherine St & Monroe St,40.711174,-73.996826,45.0
263,Elizabeth St & Hester St,40.717290,-73.996375,144.0
536,1 Ave & E 30 St,40.741444,-73.975361,137.0
259,South St & Whitehall St,40.701221,-74.012342,88.0
228,E 48 St & 3 Ave,40.754601,-73.971879,162.0
